[View in Colaboratory](https://colab.research.google.com/github/dattarajrao/keras2kubernetes/blob/master/TensorFlow_Keras_examples.ipynb)

In [5]:
import tensorflow as tf
cifar = tf.keras.datasets.cifar10

(x_train, y_train),(x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    
  tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(64, (3, 3)),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Dropout(0.25),
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
50000/50000 [==============================] - 213s 4ms/step - loss: 1.4501 - acc: 0.4731


Epoch 2/5
50000/50000 [==============================] - 216s 4ms/step - loss: 1.0726 - acc: 0.6172


Epoch 3/5
50000/50000 [==============================] - 216s 4ms/step - loss: 0.9115 - acc: 0.6804


Epoch 4/5
50000/50000 [==============================] - 216s 4ms/step - loss: 0.8028 - acc: 0.7177


Epoch 5/5
50000/50000 [==============================] - 215s 4ms/step - loss: 0.7178 - acc: 0.7476


10000/10000 [==============================] - 13s 1ms/step


[0.7826434622764588, 0.7306]

In [0]:
model.save('my_cifar_cnn.h5')

In [22]:
from tensorflow.keras.models import load_model
import numpy as np

newmodel = load_model('my_cifar_cnn.h5')
prediction = newmodel.predict(np.expand_dims(x_test[0], axis=0))
print(y_test[0])
print(np.argmax(prediction))

[3]
3


In [36]:
!pip install boto3

    100% |████████████████████████████████| 133kB 2.4MB/s 
    100% |████████████████████████████████| 4.4MB 1.7MB/s 
    100% |████████████████████████████████| 61kB 16.9MB/s 
    100% |████████████████████████████████| 552kB 15.3MB/s 


In [40]:
import boto3

AWS_ACCESS_KEY = ''
AWS_ACCESS_SECRET_KEY = ''

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_ACCESS_SECRET_KEY)

# Call S3 to list current buckets
response = s3.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket['Name'] for bucket in response['Buckets']]

# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['aws-logs-772290243791-us-east-1', 'dattaraj-lambda', 'dattaraj-models', 'dattaraj-public', 'elasticbeanstalk-ap-southeast-1-772290243791', 'elasticbeanstalk-us-east-1-772290243791']


In [44]:
# Upload a new file
s3 = boto3.resource('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_ACCESS_SECRET_KEY)
data = open('my_cifar_cnn.h5', 'rb')
s3.Bucket('dattaraj-models').put_object(Key='my_cifar_cnn.h5', Body=data)

s3.Object(bucket_name='dattaraj-models', key='my_cifar_cnn.h5')